In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen

d = {
    # 't01_sparse_baseline': ['181883'],
    't02_frame_level_test': ['181432'],
    # 's04_loss_mask': ['181574'],
    # 's06_head_loss': ['179459'],
    # 's03_sparse_baseline': ['181883']
    
    's01_baseline': ['181102'],
    's05_learn_t': ['181654'],
    'a00_baseline_10k_iter': ['182943'],
    'a00_baseline_90k_iter': ['183039'],
    'a00_baseline_100k_iter': ['182563'],
    
    'a01_more_frame_70k_iter': ['183217'],
    'a01_more_frame_90k_iter': ['183227'],
    'a01_more_frame_100k_iter': ['183278'],
     
    'a02_crop_aug_60k_iter': ['183647'],
    'a02_crop_aug_80k_iter': ['183655'],
    
    'a03_re_60k_iter': ['183854'],
    'a03_re_100k_iter': ['183865'],
     
    'a04_guassian_blur_20k_iter': ['183870'],
    'a04_guassian_blur_90k_iter': ['183888'],
     
    'a05_td_10k_iter': ['184578'],
    'a05_td_30k_iter': ['184597'],
     
    'a06_n_views_10k_iter': ['184670'],
     
    'a07_m_views_100k_iter': ['184155'],
    'a07_m_views_90k_iter': ['184310'],
    'a07_m_views_60k_iter': ['184275'],
     
    'a08_tdn_30k_iter': ['185114'],
     

    'a10_s3d_20k_iter': ['185550'],
     
    'skp0': ['183770'],
     

    'a12_2n_views_40k_iter': ['186048'],
    'a12_2n_views_60k_iter': ['186075'],
    'a12_2n_views_80k_iter': ['186109'],
    'a12_2n_views_40k_iter_white': ['186528'],
     
    'a14_2nt_w_20k_iter': ['186516'],
    'a14_2nt_w_40k_iter': ['186518'],
    'a14_2nt_w_50k_iter': ['186527'],
     

    'a17_moco_base_2k_iter': ['186871'],
    'a17_moco_base_12k_iter': ['186878'],
    'a17_moco_base_30k_iter': ['186906'],
    'a17_moco_base_40k_iter': ['186959'],
    'a17_moco_base_50k_iter': ['186968'],
    
    'a17_moco_t_2k_iter': ['187264'],
    'a17_moco_t_12k_iter': ['187466'],
    'a17_moco_t_20k_iter': ['187284'],
    'a17_moco_t_36k_iter': ['187474'],
    'a17_moco_t_50k_iter': ['187375'],
     
    'a17_moco_t_50k_encoder_k_iter': ['187388'],
     
    'a18_sample_6k_iter': ['187063'],
    'a18_sample_12k_iter': ['187150'],
    'a18_sample_20k_iter': ['187081'],
    'a18_sample_42k_iter': ['187113'],
    'a18_sample_50k_iter': ['187120'],
    
    'a19_moco_base_62k_iter': ['187599'],
    'a19_moco_base_72k_iter': ['187598'],
    'a19_moco_base_80k_iter': ['187645'],
    'a19_moco_base_90k_iter': ['187654'],
    
     
    
         
    'swav_3000': ['186356'],
}


import ssl
context = ssl._create_unverified_context()

for name in d:
    index = d[name][0]
    url = f'https://www.aicrowd.com/challenges/multi-agent-behavior-challenge-2022/problems/mabe-2022-ant-beetles-video-data/submissions/{index}'
    # From: https://mofanpy.com/tutorials/data-manipulation/scraping/beautifulsoup-css/
    # if has Chinese, apply decode()
    html = urlopen(url, context=context).read().decode('utf-8')
    soup = BeautifulSoup(html, features='lxml')
    divs = soup.find_all('div', {'class': 'mr-4'})
    for div in divs:
        span = div.find('span')
        d[name].append(span.get_text().replace('\n', ''))
    d[name].pop(3)
    d[name].pop(-1)
columns = [
    'INDEX',
    'RANK_ALL',
    'MEAN_F1',
    *['SCORE_' + str(i) for i in range(1, 14)],
    'MSE',
    'RANK_SELF',
    *['RANK_' + str(i) for i in range(1, 13)],
    'PRIVATE_LEADERBOARD',
]
df = pd.DataFrame.from_dict(d, orient='index', columns=columns)
columns_vis = [
    'INDEX',
    'RANK_ALL',
    'MEAN_F1',
    *['SCORE_' + str(i) for i in range(1, 14)],
    'MSE',
    'RANK_SELF',
    'PRIVATE_LEADERBOARD',
    *['RANK_' + str(i) for i in range(1, 13)],
]
df = df[columns_vis]
df['MEAN_F1'] = df['MEAN_F1'].astype(float)
df['SCORE_13'] = df['SCORE_13'].astype(float)
df['RANK_SELF'] = df['RANK_SELF'].astype(float)
df['PRIVATE_LEADERBOARD'] = df['PRIVATE_LEADERBOARD'].astype(int)

# df = df.sort_values('PRIVATE_LEADERBOARD', ascending=True)
df = df.sort_values('RANK_SELF', ascending=True)

df.to_csv('./rank.csv')
df.style.highlight_max(axis=0, subset=[*['SCORE_' + str(i) for i in range(1, 14) if i not in [2, 4, 9]]])